In [ ]:
import sys, os
from os.path import join, dirname, abspath
import matplotlib.pyplot as plt
from matplotlib.pyplot import Figure, Axes
import numpy as np
import networkx as nx
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from directions import *
from string import ascii_uppercase
plt.rcParams.update({
    "text.usetex": False,
    "ytick.minor.visible":False,
    "xtick.minor.visible":False,
    'xtick.direction': "in",
    'ytick.direction': "in"
})

outdir = "out"
os.makedirs(outdir,exist_ok=True)
def out(fname): return join(outdir,fname)
def savefig(plot_name): 
    plt.savefig(out(plot_name),bbox_inches="tight",dpi=250)
    
import pandas as pd
from numpy.linalg import matrix_power, eig

def arr_to_latex(M):
    return '$$\n' + r'\begin{bmatrix}' + '\n' + (r'\\' + '\n').join('&'.join(str(x) for x in row) for row in M) + '\n' + r'\end{bmatrix}' + '\n' +'$$'

def vec_to_latex(x,round=3):
    return '$$\n' + r'\begin{bmatrix}' + '\n' + (r' \\ ').join(str(np.round(v,round)) for v in x) + '\n' + r'\end{bmatrix}' + '\n' +'$$'

from car import Car
from tiles import Road, Exit, Onramp
from world import World
from traffic_signals import Stoplight

In [ ]:
def run_test_of_offset(offset):
    offset = int(np.round(offset))
    straight = [Onramp(0,2,E)]
    for i in range(19):
        straight.append(Road(i+1,2,E))
    straight[-1]=Exit(i,2,E)
    w = World(tiles=straight, cars=[])
    w.add_stoplight(Stoplight(w.tiles[8,2],direction=6,period=10,offset=offset))
    w.add_stoplight(Stoplight(w.tiles[3,2],direction=6,period=10))
    mean_car_exit_rate = []
    for i in range(5):
        w.run(500,draw=False)
        exit = straight[-1]
        ncars = []
        for _, cars in exit.exited.items():
            ncars.append(len(cars))
        mean_car_exit_rate.append(np.mean(ncars))
        w.reset()
    return 1/np.mean(mean_car_exit_rate)

In [ ]:
run_test_of_offset(5)

In [ ]:
straight1 = [Onramp(0,2,E)]
for i in range(19):
    straight1.append(Road(i+1,2,E))
straight1[-1]=Exit(i,2,E)

straight2 = [Onramp(0,0,E)]
for i in range(19):
    straight2.append(Road(i+1,0,E))
straight2[-1]=Exit(i,0,E)

w = World(tiles=straight1+straight2,cars=[])

w.add_stoplight(Stoplight(w.tiles[8,2],direction=6,period=10,offset=5))
w.add_stoplight(Stoplight(w.tiles[3,2],direction=6,period=10))

w.add_stoplight(Stoplight(w.tiles[3,0],direction=6,period=10))
w.add_stoplight(Stoplight(w.tiles[8,0],direction=6,period=10))
w.draw(markersize=10)

In [ ]:
w.run(100,draw=True,outpath=out("synctest.gif"),markersize=10)

In [ ]:
nums = []
ts = []
for t, packet in enumerate(w.car_info_packets):
    car_locs = np.zeros((w.max_y+1,w.max_x+1))
    car_locs[packet[:,2],packet[:,1]] = 1
    num_cars = np.sum(car_locs[:,:])
    nums.append(num_cars)
    ts.append(t)
plt.plot(ts,nums)
plt.title("Traffic in Intermediate Stretch")
plt.xlabel("t")
plt.ylabel("Cars in stretch")
savefig("traffic_in_synctest.png")

In [ ]:
exit = straight[-1]
ts, ncars = [0], [0]
for t, cars in exit.exited.items():
    ts.append(t)
    ncars.append(ncars[-1]+len(cars))

plt.plot(ts,ncars)

In [ ]:
exit.exited.items()

In [ ]:
# straight3 = [Onramp(0,0,E)]
# for i in range(19):
#     straight2.append(Road(i+1,0,E))
# straight3[-1]=Exit(i,0,E)
# w2 = World(tiles=straight2,cars=[])
# w2.add_stoplight(Stoplight(w2.tiles[3,0],direction=6,period=10))
# w2.add_stoplight(Stoplight(w2.tiles[8,0],direction=6,period=10))
# w2.draw(markersize=10)

In [ ]:
# w2.run(1000,draw=False,markersize=10)

In [ ]:
exit1 = straight1[-1]
ts1, ncars1 = [0], [0]
for t, cars in exit1.exited.items():
    ts1.append(t)
    ncars1.append(ncars1[-1]+len(cars))

ts2, ncars2 = [0], [0]
exit2 = straight2[-1]
for t, cars in exit2.exited.items():
    ts2.append(t)
    ncars2.append(ncars2[-1]+len(cars))

plt.plot(ts1,ncars1)
plt.plot(ts2,ncars2)
plt.title("Cars Exiting")
plt.xlabel("t")
plt.ylabel("NCars")
savefig("synctest_traffic.png")

In [ ]:
plt.plot(ts1,np.array(ncars1)-np.array(ncars2))